## 3-1. 제목 (Markdown)

In [1]:
# Assignment 5 — Inference (KoBART Summarization)

#이 노트북은 학습된 KoBART 한국어 요약 모델을 로드하여
#실제 텍스트를 요약하는 예시를 제공합니다.

## 3-2. 환경 설정 & 모델 로드

In [2]:
!pip install -q transformers sentencepiece

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from google.colab import drive
drive.mount("/content/drive")

# 학습/평가에서 사용한 모델 경로(또는 HuggingFace Hub ID)
MODEL_DIR = "/content/drive/MyDrive/boncahier/models/kobart_ko_news"
print("MODEL_DIR:", MODEL_DIR)  # 필요에 따라 변경

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
device

Mounted at /content/drive
MODEL_DIR: /content/drive/MyDrive/boncahier/models/kobart_ko_news


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


'cuda'

## 3-3. 요약 함수 정의

In [3]:
MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 128

def summarize(text, max_source_length=MAX_SOURCE_LENGTH, max_target_length=MAX_TARGET_LENGTH):
    inputs = tokenizer(
        text,
        max_length=max_source_length,
        truncation=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # ✅ KoBART/BART는 token_type_ids 사용 안 함 → 제거
    inputs.pop("token_type_ids", None)

    # ✅ GPU 사용 시 텐서를 모델 디바이스로 이동
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=max_target_length,
            num_beams=6,
            length_penalty=1.0,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

## 3-4. 예시 1–3: 실제 뉴스 문장 or 강의 스타일 텍스트

In [4]:
# 예시 1: 네이버 뉴스 스타일 문장

text1 = """
정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다.
특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고
물류비 지원과 임시선박 투입 등을 추진하기로 했습니다.
원자재 가격 상승과 글로벌 경기 둔화로 우리 경제의 대외 리스크가 커지는 상황에서,
정부는 수출 증가세 지속이 한국 경제 회복의 핵심이라고 보고 있습니다.
"""

print("=== 예시 1 ===")
print("[원문]\n", text1.strip(), "\n")
print("[요약]\n", summarize(text1))

# 예시 2: “강의 슬라이드” 느낌의 텍스트

text2 = """
본 강의에서는 지도학습과 비지도학습의 차이를 설명한다.
지도학습은 정답 레이블이 있는 데이터를 이용하여 입력과 출력 간의 관계를 학습하는 방식이며,
대표적인 알고리즘으로는 선형 회귀, 로지스틱 회귀, 서포트 벡터 머신, 신경망 등이 있다.
반면 비지도학습은 정답 레이블 없이 데이터의 패턴과 군집 구조를 찾는 방식으로,
군집화와 차원 축소가 대표적인 예이다.
"""

print("=== 예시 2 ===")
print("[원문]\n", text2.strip(), "\n")
print("[요약]\n", summarize(text2))

# 예시 3: 간단한 합성 예시

text3 = """
인공지능 기반 요약 시스템은 학생들이 방대한 강의 자료를 빠르게 복습하는 데 도움을 줄 수 있다.
예를 들어, PDF나 PPT 형식의 수업 자료를 업로드하면,
모델이 각 슬라이드의 핵심 문장을 추출하고 짧은 한국어 요약을 생성한다.
이를 통해 시험 기간에 효율적으로 내용을 정리하고,
외국어로 작성된 슬라이드라면 번역 기능과 함께 사용할 수도 있다.
"""

print("=== 예시 3 ===")
print("[원문]\n", text3.strip(), "\n")
print("[요약]\n", summarize(text3))


=== 예시 1 ===
[원문]
 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다.
특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고
물류비 지원과 임시선박 투입 등을 추진하기로 했습니다.
원자재 가격 상승과 글로벌 경기 둔화로 우리 경제의 대외 리스크가 커지는 상황에서,
정부는 수출 증가세 지속이 한국 경제 회복의 핵심이라고 보고 있습니다. 

[요약]
 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했으며 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고
물류비 지원과 임시선박 투입 등을 추진하기로 하는 등, 수출 증가세 지속이 한국 경제 회복의 핵심이라고 보고 있습니다.  
정부는 수출 증가세가 한국 경제회복의 핵심으로 보고 있으며, 특히 원자재 가격 상승과 글로벌 경기 둔화로 우리 경제의 대외 리스크가 커지는 상황에서,
정부는 수출이 수출 증가세를 지속이한국 경제 회복에 핵심이라고 생각하고 있습니다.
원자재 가격 등 국내의 주요 리스크가 높은 상황에서,정
=== 예시 2 ===
[원문]
 본 강의에서는 지도학습과 비지도학습의 차이를 설명한다.
지도학습은 정답 레이블이 있는 데이터를 이용하여 입력과 출력 간의 관계를 학습하는 방식이며,
대표적인 알고리즘으로는 선형 회귀, 로지스틱 회귀, 서포트 벡터 머신, 신경망 등이 있다.
반면 비지도학습은 정답 레이블 없이 데이터의 패턴과 군집 구조를 찾는 방식으로,
군집화와 차원 축소가 대표적인 예이다. 

[요약]
 본 강의에서는 지도학습과 비지도학습의 차이를 설명하며 지도학습은 정답 레이블이 있는 데이터를 이용하여 입력과 출력 간의 관계를 학습하는 방식으로,
대표적인 알고리즘으로는 선형 회귀, 로지스틱 회귀 서포트 벡터 머신, 신경망 등이 있다고 하였다.반면 비지도 학습은 데이터의 패턴과 군집 구조를 찾는 방식으로, 군집화와 차원 축소가 대표적인 예이다.
=== 예시 3 ===
[원